In [9]:
# ============================================================
# Multi-Agent Financial Advisory System (dynamic ticker; 2000-token cap at call time)
# ============================================================

from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# ------------------------------------------------------------
# Logging
# ------------------------------------------------------------
logging.getLogger("strands").setLevel(logging.INFO)

# ------------------------------------------------------------
# Token cap to try at CALL TIME (not constructor)
# We'll try several common parameter names safely.
# ------------------------------------------------------------
TOKEN_CAP = 2000

def invoke_agent(agent: Agent, prompt: str, max_tokens: int = TOKEN_CAP) -> str:
    """
    Call an Agent with a conservative 2000-token cap.
    Tries several common kwarg names, then falls back to plain call.
    """
    # Try common inference kwargs in descending likelihood:
    for kwargs in (
        {"max_output_tokens": max_tokens},               # Anthropic-like wrappers
        {"max_tokens": max_tokens},                      # OpenAI-style
        {"generation_config": {"max_output_tokens": max_tokens}},  # some SDKs
        {"inference_params": {"max_tokens": max_tokens}},          # generic
    ):
        try:
            return str(agent(prompt, **kwargs))
        except TypeError:
            continue
        except Exception as e:
            return f"Agent invocation error: {e}"

    # Last-ditch: call without any kwargs
    try:
        return str(agent(prompt))
    except Exception as e:
        return f"Agent invocation error (no-kwargs): {e}"

# ------------------------------------------------------------
# Web Search Tool (DuckDuckGo via ddgs)
# ------------------------------------------------------------
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """
    Search the web to get updated information.
    Args:
        keywords (str): The search query.
        region (str): Region code like 'wt-wt', 'us-en', 'uk-en', etc.
        max_results (int | None): Max number of results.
    Returns:
        str: Results or error string (agents will read/parse as needed).
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

# ============================================================
# Specialist Agents (Professional Names) — all include websearch
# ============================================================

MarketIntelligenceAgent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""You are a specialized Market Intelligence Agent within a financial advisory system.
Your role is to research and analyze market data, news, and filings for a given ticker symbol using web search.

## Responsibilities:
1. Perform web-based research using DuckDuckGo (via the `websearch` tool)
2. Extract insights from SEC filings, reputable news, performance context, and analyst commentary
3. Provide a structured, objective market summary with sources
4. Always include this disclaimer: "This analysis is for educational purposes only and does not constitute financial advice."
""",
    tools=[websearch],
)

StrategyArchitectAgent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""You are a specialized Strategy Architect Agent within a financial advisory system.
Your role is to develop tailored trading strategies based on user preferences, risk tolerance, and market analysis data.

## Responsibilities:
1. Generate at least 5 distinct trading strategies
2. Align strategies with the user's risk attitude (Conservative, Moderate, Aggressive)
3. Match strategies to the user's investment horizon (Short-term, Medium-term, Long-term)
4. Provide clear rationale and risk assessment for each strategy

Always include the disclaimer: "These strategies are for educational purposes only and do not constitute financial advice."
""",
    tools=[websearch],
)

ExecutionPlannerAgent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""You are a specialized Execution Planner Agent within a financial advisory system.
Your role is to translate trading strategies into actionable execution guidance with detailed risk management protocols.

## Responsibilities:
1. Create detailed execution plans covering all phases of implementation
2. Specify order types, position sizing, and execution timing
3. Define comprehensive risk controls and mitigation steps
4. Provide step-by-step tactical guidance

Always include the disclaimer: "This execution plan is for educational purposes only and does not constitute financial advice."
""",
    tools=[websearch],
)

RiskAssessorAgent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""You are a specialized Risk Assessor Agent within a financial advisory system.
Your role is to provide comprehensive risk analysis of the proposed financial plan, evaluating consistency across market analysis,
trading strategies, execution plans, and user preferences.

## Responsibilities:
1. Evaluate all relevant risk categories comprehensively
2. Assess alignment with the user's stated risk tolerance and timeline
3. Identify misalignments or areas of concern
4. Provide actionable risk mitigation recommendations

Always include the disclaimer: "This risk assessment is for educational purposes only and does not constitute financial advice."
""",
    tools=[websearch],
)

# ============================================================
# Tools that Delegate to Specialists (typed; dynamic inputs)
# ============================================================
@tool
def market_intel_tool(ticker: str, lookback_days: int = 7) -> str:
    """
    Delegates market research & analysis to MarketIntelligenceAgent.
    Dynamic ticker + lookback. Examples: ticker='AAPL', lookback_days=7
    """
    prompt = (
        f"Research {ticker} for the last {lookback_days} days. "
        f"Use `websearch` to find SEC filings, reputable news, and analyst commentary. "
        f"Provide sources and a concise market summary. "
        f'End with: "This analysis is for educational purposes only and does not constitute financial advice."'
    )
    return invoke_agent(MarketIntelligenceAgent, prompt, max_tokens=TOKEN_CAP)

@tool
def strategy_architect_tool(ticker: str, risk_attitude: str, horizon: str) -> str:
    """
    Delegates strategy generation to StrategyArchitectAgent.
    risk_attitude ∈ {Conservative, Moderate, Aggressive}
    horizon ∈ {Short-term, Medium-term, Long-term}
    """
    prompt = (
        f"Based on market context for {ticker}, generate 5 distinct strategies. "
        f"Align to risk attitude: {risk_attitude}; and horizon: {horizon}. "
        f"For each: objective, thesis, entry/exit, risk, and expected conditions. "
        f'End with: "These strategies are for educational purposes only and do not constitute financial advice."'
    )
    return invoke_agent(StrategyArchitectAgent, prompt, max_tokens=TOKEN_CAP)

@tool
def execution_planner_tool(ticker: str, strategy_summary: str) -> str:
    """
    Delegates execution planning to ExecutionPlannerAgent.
    Pass in a short 'strategy_summary' text from StrategyArchitectAgent output.
    """
    prompt = (
        f"Convert the following strategy summary for {ticker} into a detailed execution plan:\n\n"
        f"{strategy_summary}\n\n"
        f"Include order types, sizing, timing windows, and risk controls (stops, limits, hedges). "
        f'End with: "This execution plan is for educational purposes only and does not constitute financial advice."'
    )
    return invoke_agent(ExecutionPlannerAgent, prompt, max_tokens=TOKEN_CAP)

@tool
def risk_assessor_tool(ticker: str, market_summary: str, strategies_summary: str, execution_plan: str) -> str:
    """
    Delegates comprehensive risk evaluation to RiskAssessorAgent.
    Accepts prior outputs to assess alignment and risks.
    """
    prompt = (
        f"Evaluate overall risk for {ticker}. Consider:\n"
        f"- Market summary:\n{market_summary}\n\n"
        f"- Strategies summary:\n{strategies_summary}\n\n"
        f"- Execution plan:\n{execution_plan}\n\n"
        f"Identify misalignments, key risks, and actionable mitigations. "
        f'End with: "This risk assessment is for educational purposes only and does not constitute financial advice."'
    )
    return invoke_agent(RiskAssessorAgent, prompt, max_tokens=TOKEN_CAP)

# ============================================================
# Orchestrator (Advisory Orchestrator Agent)
# ============================================================
FINANCIAL_COORDINATOR_SYSTEM_PROMPT = """You are the Financial Coordinator Agent — the lead orchestrator in a comprehensive financial advisory system.
Coordinate specialists, request missing inputs (ticker, risk attitude, horizon), summarize outputs, and keep users informed.

Always include: "**Important:** This is for educational purposes only and does not constitute financial advice."
Keep responses concise and under ~2000 tokens when possible.
"""

ORCHESTRATOR_TOOLS = [
    market_intel_tool,
    strategy_architect_tool,
    execution_planner_tool,
    risk_assessor_tool,
]

AdvisoryOrchestratorAgent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=FINANCIAL_COORDINATOR_SYSTEM_PROMPT,
    callback_handler=None,
    tools=ORCHESTRATOR_TOOLS,
)

# ============================================================
# Diagnostics / Sanity Prints
# ============================================================
print("✅ Multi-Agent Financial Advisory System initialized")
print("   Orchestrator: AdvisoryOrchestratorAgent")
print(f"   Tools mounted: {len(ORCHESTRATOR_TOOLS)}")
for i, t in enumerate(ORCHESTRATOR_TOOLS, 1):
    t_name = getattr(t, "__name__", getattr(t, "name", "unknown_tool"))
    print(f"   {i}. {t_name}")

# ============================================================
# Example Orchestration (dynamic ticker)
# ============================================================
# Example flow (uncomment to run end-to-end):
# ticker = "AAPL"
# market = market_intel_tool(ticker=ticker, lookback_days=7)
# strategies = strategy_architect_tool(ticker=ticker, risk_attitude="Moderate", horizon="Medium-term")
# execution = execution_planner_tool(ticker=ticker, strategy_summary=strategies[:2000])  # pass trimmed summary if huge
# risk = risk_assessor_tool(ticker=ticker, market_summary=market[:2000], strategies_summary=strategies[:2000], execution_plan=execution[:2000])
# print("\n--- Market ---\n", market)
# print("\n--- Strategies ---\n", strategies)
# print("\n--- Execution ---\n", execution)
# print("\n--- Risk ---\n", risk)


✅ Multi-Agent Financial Advisory System initialized
   Orchestrator: AdvisoryOrchestratorAgent
   Tools mounted: 4
   1. market_intel_tool
   2. strategy_architect_tool
   3. execution_planner_tool
   4. risk_assessor_tool


In [10]:
# ============================================================
# Example Orchestration (dynamic ticker)
# ============================================================
# Example flow (uncomment to run end-to-end):
ticker = "AMZN"
market = market_intel_tool(ticker=ticker, lookback_days=7)
strategies = strategy_architect_tool(ticker=ticker, risk_attitude="Moderate", horizon="Medium-term")
execution = execution_planner_tool(ticker=ticker, strategy_summary=strategies[:2000])  # pass trimmed summary if huge
risk = risk_assessor_tool(ticker=ticker, market_summary=market[:2000], strategies_summary=strategies[:2000], execution_plan=execution[:2000])
print("\n--- Market ---\n", market)
print("\n--- Strategies ---\n", strategies)
print("\n--- Execution ---\n", execution)
print("\n--- Risk ---\n", risk)

I'll research Amazon (AMZN) for the last 7 days using the websearch tool to gather information from SEC filings, news sources, and analyst commentary. Let me do that for you now.
Tool #1: websearch
Let me search for more recent news and analyst commentary about Amazon:
Tool #2: websearch
Let me gather more specific details about Amazon's recent performance and analyst ratings:
Tool #3: websearch

Tool #4: websearch

Tool #5: websearch
Based on my research, here's a concise market summary for Amazon (AMZN) over the last 7 days:

# Amazon (AMZN) Market Summary - Last 7 Days

## Stock Performance
Amazon shares have shown moderate movement in the past week, maintaining their position as a key member of the "Magnificent Seven" tech stocks. Recent trading indicates the stock has been performing in line with broader tech sector trends in October 2025.

## Recent Developments
1. **New Product Launch**: Amazon recently rolled out a new private label called "Amazon Grocery," which consolidates i